In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import the dataset

## Import the main dataset

In [2]:
hotel = pd.read_csv("Hotel_Reviews.csv")

In [3]:
len(hotel)

515738

In [4]:
len(hotel["Hotel_Name"].unique())

1492

In [5]:
len(hotel["Hotel_Address"].unique())

1493

In [6]:
hotel["Reviewer_Nationality"].value_counts()

 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Cape Verde                        1
 Northern Mariana Islands          1
 Tuvalu                            1
 Guinea                            1
 Palau                             1
Name: Reviewer_Nationality, Length: 227, dtype: int64

In [7]:
len(hotel["Hotel_Address"].value_counts())

1493

In [8]:
len(hotel["Reviewer_Nationality"].unique())

227

In [9]:
len(hotel["Hotel_Name"].unique())

1492

### Trim the space of Review_Nationality Column for further merging

In [10]:
hotel["Hotel_Address"] = hotel["Hotel_Address"].apply(lambda x: x.strip())
hotel["Hotel_Name"] = hotel["Hotel_Name"].apply(lambda x: x.strip())
hotel["Negative_Review"] = hotel["Negative_Review"].apply(lambda x: x.strip())
hotel["Reviewer_Nationality"] = hotel["Reviewer_Nationality"].apply(lambda x: x.strip())
hotel["Positive_Review"] = hotel["Positive_Review"].apply(lambda x: x.strip())

### Drop the rows of data whose Nationality column is empty

In [11]:
len(hotel[(hotel["Positive_Review"] == '') & (hotel["Negative_Review"] == '')])

59

In [12]:
len(hotel[hotel["Positive_Review"] == ''])

183

In [13]:
len(hotel[hotel["Negative_Review"] == ''])

849

In [14]:
len(hotel[(hotel["Reviewer_Nationality"] == '')])

523

In [17]:
hotel.drop(hotel[hotel["Reviewer_Nationality"] == ''].index, axis=0, inplace=True)

In [18]:
hotel.drop(hotel[(hotel["Positive_Review"] == '') & (hotel["Negative_Review"] == '')].index, axis=0, inplace=True)

In [19]:
len(hotel)

515156

In [15]:
len(hotel[hotel["Negative_Review"] == "No Negative"])

127890

In [20]:
hotel.loc[hotel['Negative_Review'] == 'No Negative', 'Negative_Review'] = None

In [21]:
hotel[hotel['Negative_Review'] == "No Negative"]

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng


In [16]:
len(hotel[hotel["Positive_Review"] == "No Positive"])

35946

In [22]:
hotel.loc[hotel['Positive_Review'] == 'No Positive', 'Positive_Review'] = None

In [25]:
hotel[hotel['Positive_Review'] == "No Positive"]

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng


### Test and see whether is there any other columns has null value

In [26]:
null_columns = hotel.columns[hotel.isnull().any()]
null_columns

Index(['Negative_Review', 'Positive_Review', 'lat', 'lng'], dtype='object')

In [27]:
columns_with_null = hotel.isnull().any()

# 輸出具有空值的欄位名稱
print("具有空值的欄位：")
print(columns_with_null[columns_with_null].index.tolist())

具有空值的欄位：
['Negative_Review', 'Positive_Review', 'lat', 'lng']


In [28]:
len(hotel[hotel["lat"].isnull()])

3265

In [29]:
len(hotel[hotel["lng"].isnull()])

3265

In [33]:
# text = hotel["Tags"][1].replace("[", "").replace("]", "").replace("'", "")
# words = text.split(", ")
# import pandas as pd
def process_tags(tag):
    tag = tag.replace("[", "").replace("]", "").replace("'", "")  # 移除特殊符號
    tag_list = tag.split(", ")  # 利用split()函數分割字符串，形成列表
    return tag_list

# 新增tag_list欄位
hotel['Tags_list'] = hotel['Tags'].apply(lambda x: process_tags(x))

In [ ]:


# 將tag_list列的列表元素展開成新的行
exploded_tags = hotel['Tags_list'].explode()

# 計算每個元素出現的次數
tag_counts = exploded_tags.value_counts()
print(tag_counts)


In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [ ]:
from wordcloud import WordCloud

# Assuming word_counts is your Pandas Series containing words and their counts

# Convert the Series to a dictionary for WordCloud
wordcloud_dict = tag_counts.to_dict()

# Generate the WordCloud object
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(wordcloud_dict)

# Display the WordCloud using matplotlib
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Remove axis
plt.savefig('wordcloud_image.png', dpi=300, bbox_inches='tight')
plt.show()

## Import the country-region matching list

In [30]:
region = pd.read_csv("region_list.csv")

## Merge the hotel dataset with region matching list

In [34]:
merged_df = pd.merge(hotel, region, left_on='Reviewer_Nationality', right_on='name', how='left')

In [36]:
merged_df.to_csv("hotel_review_newregion.csv")

In [35]:
merged_df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Tags_list,name,region,sub-region
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available ...,397,1403,Only the park outside of the hotel was beautiful,...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,"[ Leisure trip , Couple , Duplex Double Room...",Russia,Europe,Eastern Europe
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,8/3/2017,7.7,Hotel Arena,Ireland,None,0,1403,No real complaints the hotel was great great l...,...,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,"[ Leisure trip , Couple , Duplex Double Room...",Ireland,Europe,Northern Europe
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficult...,42,1403,Location was good and staff were ok It is cute...,...,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,"[ Leisure trip , Family with young children ,...",Australia,Oceania,Australia and New Zealand
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk bar...,210,1403,Great location in nice surroundings the bar an...,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,"[ Leisure trip , Solo traveler , Duplex Doub...",United Kingdom,Europe,Northern Europe
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line yo...,140,1403,Amazing location and building Romantic setting,...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,"[ Leisure trip , Couple , Suite , Stayed 2 ...",New Zealand,Oceania,Australia and New Zealand
